In [ ]:
# Cell 1: Import libraries
import os
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as T
import random
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
# Cell 2: Settings
INPUT_FOLDER = "path/to/your/images"  # CHANGE THIS
OUTPUT_FOLDER = "output_balanced"     # CHANGE THIS
IMAGE_SIZE = 224
TARGET_COUNT = 30  # Target images per class
SEED = 42

random.seed(SEED)
np.random.seed(SEED)

In [ ]:
# Cell: Clear output folders
import shutil

# Define folders to clear
folders_to_clear = [
    f"{OUTPUT_FOLDER}/augmented",
    f"{OUTPUT_FOLDER}/resized", 
    f"{OUTPUT_FOLDER}/final"
]

# Clear each folder
print("Clearing output folders...")
for folder in folders_to_clear:
    if os.path.exists(folder):
        # Remove all contents
        shutil.rmtree(folder)
        # Recreate empty folder
        os.makedirs(folder)
        print(f"  ✓ Cleared: {folder}")
    else:
        # Create if doesn't exist
        os.makedirs(folder)
        print(f"  ✓ Created: {folder}")

print("\nAll output folders cleared and ready!")

In [ ]:
# Cell 3: Create folders
os.makedirs(f"{OUTPUT_FOLDER}/augmented", exist_ok=True)
os.makedirs(f"{OUTPUT_FOLDER}/resized", exist_ok=True)
os.makedirs(f"{OUTPUT_FOLDER}/final", exist_ok=True)

In [ ]:
# Cell 4: Basic functions
def get_class_label(filename):
    """Get class from filename (e.g., '0p', '5p', '100p')"""
    match = re.search(r'(\d+)[pP]', filename)
    if match:
        return f"{match.group(1)}p"
    return "unknown"

def get_numeric_label(filename):
    """Extract numeric value from filename for regression"""
    match = re.search(r'(\d+)[pP]', filename)
    if match:
        return float(match.group(1))
    return 0.0

def make_square(image, size):
    """Resize image and pad to square while keeping RGB"""
    # Convert to RGB if not already
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    # Calculate new size
    w, h = image.size
    scale = size / max(w, h)
    new_w = int(w * scale)
    new_h = int(h * scale)
    
    # Resize
    resized = image.resize((new_w, new_h), Image.BILINEAR)
    
    # Create square with white background
    square = Image.new('RGB', (size, size), (255, 255, 255))
    
    # Paste in center
    x = (size - new_w) // 2
    y = (size - new_h) // 2
    square.paste(resized, (x, y))
    
    return square

In [ ]:
# Cell 5: Define augmentations
def aug1(img): return img.rotate(90, expand=True)     # 90° rotation
def aug2(img): return img.rotate(180, expand=True)    # 180° rotation  
def aug3(img): return img.rotate(270, expand=True)    # 270° rotation
def aug4(img): return img.transpose(Image.FLIP_LEFT_RIGHT)
def aug5(img): return img.transpose(Image.FLIP_TOP_BOTTOM)

augmentation_list = [aug1, aug2, aug3, aug4, aug5]

In [ ]:
# Cell 6: Group images by class
image_files = [f for f in os.listdir(INPUT_FOLDER) 
               if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp'))]

# Group by class
classes = {}
for filename in image_files:
    label = get_class_label(filename)
    if label not in classes:
        classes[label] = []
    classes[label].append(filename)

# Show counts
print("Images per class:")
for label, images in sorted(classes.items()):
    print(f"  {label}: {len(images)} images")

In [ ]:
# Cell 7: Copy originals and create augmentations
print("\nProcessing images...")

for class_label, filenames in sorted(classes.items()):
    print(f"\nClass {class_label}:")
    
    # Copy all originals first
    for filename in filenames:
        img = Image.open(os.path.join(INPUT_FOLDER, filename))
        # Keep as RGB (no binary conversion)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        
        base = os.path.splitext(filename)[0]
        img.save(f"{OUTPUT_FOLDER}/augmented/{base}_original.jpg")
    
    # Create augmentations if needed
    current_count = len(filenames)
    if current_count < TARGET_COUNT:
        needed = TARGET_COUNT - current_count
        print(f"  Creating {needed} augmentations...")
        
        for i in range(needed):
            # Pick random original
            source_file = random.choice(filenames)
            img = Image.open(os.path.join(INPUT_FOLDER, source_file))
            # Keep as RGB (no binary conversion)
            if img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Apply 3-4 random augmentations
            num_augs = random.randint(3, 4)
            selected_augs = random.sample(augmentation_list, num_augs)
            
            augmented = img
            for aug_func in selected_augs:
                augmented = aug_func(augmented)
            
            # Save
            base = os.path.splitext(source_file)[0]
            augmented.save(f"{OUTPUT_FOLDER}/augmented/{base}_aug{i}.jpg")

In [ ]:
# Cell 8: Resize all images
print("\nResizing all images...")
augmented_files = [f for f in os.listdir(f"{OUTPUT_FOLDER}/augmented") 
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

for i, filename in enumerate(augmented_files):
    if i % 50 == 0:
        print(f"  {i}/{len(augmented_files)}")
    
    img = Image.open(f"{OUTPUT_FOLDER}/augmented/{filename}")
    resized = make_square(img, IMAGE_SIZE)
    
    # Save as JPG to maintain RGB
    base = os.path.splitext(filename)[0]
    resized.save(f"{OUTPUT_FOLDER}/resized/{base}.jpg")

In [ ]:
# Cell 9: Normalize for ResNet (ImageNet normalization)
print("\nNormalizing images for ResNet...")
resized_files = [f for f in os.listdir(f"{OUTPUT_FOLDER}/resized") 
                 if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# ImageNet normalization values for ResNet
normalize = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

for i, filename in enumerate(resized_files):
    if i % 50 == 0:
        print(f"  {i}/{len(resized_files)}")
    
    # Load RGB image
    img = Image.open(f"{OUTPUT_FOLDER}/resized/{filename}")
    
    # Apply ResNet normalization
    tensor = normalize(img)
    
    # Save tensor
    base = os.path.splitext(filename)[0]
    np.save(f"{OUTPUT_FOLDER}/final/{base}.npy", tensor.numpy())
    
    # Save image for viewing
    img.save(f"{OUTPUT_FOLDER}/final/{base}.jpg")

In [ ]:
# Cell 10: Check final counts
print("\nFinal image count per class:")
final_counts = {}
for filename in os.listdir(f"{OUTPUT_FOLDER}/final"):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        label = get_class_label(filename)
        final_counts[label] = final_counts.get(label, 0) + 1

for label, count in sorted(final_counts.items()):
    print(f"  {label}: {count} images")

In [ ]:
# Cell 11: Simple Model Setup
import torch.nn as nn
import torchvision.models as models

# Create ResNet model for regression
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 1)  # Single output for regression
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(f"Model ready on {device}")

In [ ]:
# Cell 12: Simple Dataset and Data Loaders
from torch.utils.data import Dataset, DataLoader

class ImageDataset(Dataset):
    def __init__(self, folder):
        self.files = [f for f in os.listdir(folder) if f.endswith('.npy')]
        self.folder = folder
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        file = self.files[idx]
        tensor = torch.from_numpy(np.load(f"{self.folder}/{file}")).float()
        label = torch.tensor(get_numeric_label(file), dtype=torch.float32)
        return tensor, label

# Create dataset and split
dataset = ImageDataset(f"{OUTPUT_FOLDER}/final")
train_size = int(0.8 * len(dataset))
train_set, val_set = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])

train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
val_loader = DataLoader(val_set, batch_size=16)

print(f"Dataset ready: {len(train_set)} train, {len(val_set)} val samples")

In [ ]:
# Cell 13: Simple Training Loop
from sklearn.metrics import mean_absolute_error, r2_score

history = {'train_loss': [], 'val_loss': [], 'train_mae': [], 'val_mae': [], 'train_r2': [], 'val_r2': []}

for epoch in range(10):
    # Train
    model.train()
    train_loss, train_preds, train_targets = 0, [], []
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data).view(-1)  # Use view(-1) instead of squeeze()
        target = target.view(-1)       # Ensure target has same shape
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        train_preds.extend(output.detach().cpu().numpy())
        train_targets.extend(target.cpu().numpy())
    
    # Validate
    model.eval()
    val_loss, val_preds, val_targets = 0, [], []
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data).view(-1)  # Use view(-1) instead of squeeze()
            target = target.view(-1)       # Ensure target has same shape
            val_loss += criterion(output, target).item()
            val_preds.extend(output.cpu().numpy())
            val_targets.extend(target.cpu().numpy())
    
    # Calculate metrics
    train_mae = mean_absolute_error(train_targets, train_preds)
    val_mae = mean_absolute_error(val_targets, val_preds)
    train_r2 = r2_score(train_targets, train_preds)
    val_r2 = r2_score(val_targets, val_preds)
    
    # Store results
    history['train_loss'].append(train_loss/len(train_loader))
    history['val_loss'].append(val_loss/len(val_loader))
    history['train_mae'].append(train_mae)
    history['val_mae'].append(val_mae)
    history['train_r2'].append(train_r2)
    history['val_r2'].append(val_r2)
    
    print(f"Epoch {epoch+1}: Loss={val_loss/len(val_loader):.3f}, MAE={val_mae:.1f}, R²={val_r2:.3f}")

torch.save(model.state_dict(), f'{OUTPUT_FOLDER}/model.pth')
print("Training complete!")

In [ ]:
# Cell 14: Simple Results
import matplotlib.pyplot as plt

# Plot results
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

ax1.plot(history['train_loss'], label='Train')
ax1.plot(history['val_loss'], label='Val')
ax1.set_title('Loss')
ax1.legend()

ax2.plot(history['train_mae'], label='Train')
ax2.plot(history['val_mae'], label='Val')
ax2.set_title('MAE')
ax2.legend()

ax3.plot(history['train_r2'], label='Train')
ax3.plot(history['val_r2'], label='Val')
ax3.set_title('R²')
ax3.legend()

plt.show()

# Test predictions
model.eval()
with torch.no_grad():
    for data, target in val_loader:
        data, target = data.to(device), target.to(device)
        pred = model(data).view(-1)  # Use view(-1) instead of squeeze()
        target = target.view(-1)     # Ensure target has same shape
        print("Sample Predictions:")
        for i in range(min(5, len(pred))):
            print(f"Predicted: {pred[i].item():.1f}, Actual: {target[i].item():.1f}")
        break